# Grammar of Data: Pandas and SQL

## Part 1: Pandas

In [1]:
#Import Pandas library
import pandas as pd

We first need to load in our CSV file as a Pandas dataframe. The easiest way to do this is to simply use Pandas `read_csv` function. 

Let's look at the first 3 lines of the file (data/candidates.txt):

```
id|first_name|last_name|middle_name|party

33|  Joseph  |  Biden  |           |  D

36|  Samuel  |Brownback|           |  R
```

Observe that each entry is separated by a pipe '|'. Since the default separator for the `read_csv` function is a comma, we will have to specify this separator when we call the function.

In [2]:
### edTest(test_candloading) ###
#Get the data using pandas read_csv

dfcand=pd.read_csv("data/candidates.txt", "|" )
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


Next, let us also load in the "contributors_with_candidate_id.txt" file that has the people who have contributed money to candidates.

Lets again take a look at the first three lines before loading in the data:

```
id|last_name|first_name|middle_name|street_1|street_2|city|state|zip|amount|date|candidate_id

  |   Agee  |  Steven  |           |549 Laurel Branch Road||Floyd|VA|24091|500.00|2007-06-30|16

  |  Ahrens |   Don    |           |4034 Rennellwood Way||Pleasanton|CA|94566|250.00|2007-05-16|16
```

In [3]:
### edTest(test_contloading) ###

#access the data of the second txt file "contributors_with_candidate_id.txt" using the same format
dfcwci=pd.read_csv("data/contributors_with_candidate_id.txt","|")
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


You'll notice that the contributions dont have the first column, so we will need to clean things up a bit...

In [4]:
del dfcwci['id']
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


## Part 2: SQLITE

Now, lets start with Relational Databases. We use SQLITE here. Sqlite is a on-file database, as opposed to other common databases such as Oracle and Postgres, which run as different processes on your system. Python implements a standard database API over all databases. Its called DBAPI2. It works across many SQL databases.

The available attributes are constrained by a "header" tuple of attributes which set the type. We do this below, using the SQL language to set things up.

In [5]:
#lets create a schema to work on our data using SQL
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR, -- Notice that we are converting the zip from integer to string
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

Now we'll write some functions to connect to Sqlite.

### Step 1: Connect and get a DBAPI2 connection.

In [6]:
from sqlite3 import dbapi2 as sq3
from pathlib import Path
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(Path(PATHSTART) / dbfile)
    return sqlite_db

### Step 2: Set up the database with tables

In [7]:
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile) 
    db.cursor().executescript(schema)
    db.commit()
    return db

Initializing the database:

In [8]:
db=init_db("/tmp/cancont.db", ourschema)

### Step 3: Populate with Pandas

In [9]:
#Converts into a SQL Database
dfcand.to_sql("candidates", db, if_exists="append", index=False)
dfcwci.to_sql("contributors", db, if_exists="append", index=False)

### Step 4: Query the database

In [10]:
#Defining a function to make a query
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [11]:
#Defining a function to convert the ouputs to a pandas dataframe
#Import OrderedDict to remember the order in which keys were inserted
from collections import OrderedDict
def make_frame(list_of_tuples, legend):
    framelist=[]
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_dict(OrderedDict(framelist)) 

In [12]:
#Checking the columns of candidate table
candidate_cols = [e[1] for e in make_query("PRAGMA table_info(candidates);")]
candidate_cols

['id', 'first_name', 'last_name', 'middle_name', 'party']

In [13]:
#Checking the columns of contributors table
contributor_cols = [e[1] for e in make_query("PRAGMA table_info(contributors);")]
contributor_cols

['id',
 'last_name',
 'first_name',
 'middle_name',
 'street_1',
 'street_2',
 'city',
 'state',
 'zip',
 'amount',
 'date',
 'candidate_id']

Test that it works...

In [14]:
out=make_query("SELECT * FROM candidates;")
#Create a dataframe
make_frame(out, legend=candidate_cols)

,id,first_name,last_name,middle_name,party
0,16,Mike,Huckabee,None,R
1,20,Barack,Obama,None,D
2,22,Rudolph,Giuliani,None,R
3,24,Mike,Gravel,None,D
4,26,John,Edwards,None,D
5,29,Bill,Richardson,None,D
6,30,Duncan,Hunter,None,R
7,31,Dennis,Kucinich,None,D
8,32,Ron,Paul,None,R
9,33,Joseph,Biden,None,D


## Part 3: The Grammar of Data

<table>
  <tr>
    <th><b>VERB</b></th>
    <th><b>dplyr</b></th>
    <th><b>pandas</b></th>
    <th><b>SQL</b></th>
  </tr>
  <tr>
    <td>QUERY/SELECTION</td>
    <td>filter() (and slice())</td>
    <td>query() (and loc[], iloc[])</td>
    <td>SELECT WHERE</td>
  </tr>
  <tr>
    <td>SORT</td>
    <td>arrange()</td>
    <td>sort_values()</td>
    <td>ORDER BY</td>
  </tr>
  <tr>
    <td>SELECT-COLUMNS/PROJECTION</td>
    <td>select() (and rename())</td>
    <td>[](__getitem__) (and rename())</td>
    <td>SELECT COLUMN</td>
  </tr>
  <tr>
    <td>SELECT-DISTINCT</td>
    <td>distinct()</td>
    <td>unique(),drop_duplicates()</td>
    <td>SELECT DISTINCT COLUMN</td>
  </tr>
  <tr>
    <td>ASSIGN</td>
    <td>mutate() (and transmute())</td>
    <td>assign</td>
    <td>ALTER/UPDATE</td>
  </tr>
  <tr>
    <td>AGGREGATE</td>
    <td>summarise()</td>
    <td>describe(), mean(), max()</td>
    <td>None, AVG(),MAX()</td>
  </tr>
  <tr>
    <td>SAMPLE</td>
    <td>sample_n() and sample_frac()</td>
    <td>sample()</td>
    <td>implementation dep, use RAND()</td>
  </tr>
  <tr>
    <td>GROUP-AGG</td>
    <td>group_by/summarize</td>
    <td>groupby/agg, count, mean</td>
    <td>GROUP BY</td>
  </tr>
  <tr>
    <td>DELETE</td>
    <td>?</td>
    <td>drop/masking</td>
    <td>DELETE/WHERE</td>
  </tr>
</table>


Now let's learn a few of these by answering a few questions! 

### Q1: Find the columns for which the *state* is NULL

#### Q1.1: How do you do it in Pandas?

In [15]:
### edTest(test_null) ###
dfcwci_null=dfcwci[dfcwci.state.isnull()]
dfcwci_null

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


#### Q1.2: How do you do it in SQL?

In [16]:
out=make_query("SELECT * FROM contributors WHERE state IS NULL")
db_null=make_frame(out, legend=contributor_cols)
db_null

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,None,LAGE KAART 77,None,BRASSCHATT,None,2930,-500,2008-11-20,35


### Q2: Sort contributors by the amount they contributed in ascending and descending order 

#### Q2.1 How do you do it in Pandas?

##### First, let's see it in ascending order.

In [17]:
dfcwci_asc=dfcwci.sort_values("amount")
dfcwci_asc

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
90,Kazor,Christopher,M,707 Spindletree ave,NaN,Naperville,IL,60565,-2592.0,2008-04-21,32
72,BRUNO,JOHN,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300.0,2008-03-06,22
64,BURKE,DONALD,J.,12 LOMPOC,NaN,RANCHO SANTA MARGA,CA,926881817,-2300.0,2008-03-11,22
73,BRUNO,IRENE,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300.0,2008-03-06,22
74,BROWN,TIMOTHY,J.,26826 MARLOWE COURT,NaN,STEVENSON RANCH,CA,913811020,-2300.0,2008-03-06,22
...,...,...,...,...,...,...,...,...,...,...,...
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.0,2007-09-30,20
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.0,2007-06-12,16
135,ABRAMOWITZ,NIRA,NaN,411 HARBOR ROAD,NaN,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37


##### Next, sort the same in descending order.

In [18]:
### edTest(test_desc) ###
dfcwci_desc=dfcwci.sort_values("amount" , ascending=False ) 
dfcwci_desc

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.0,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.0,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20
...,...,...,...,...,...,...,...,...,...,...,...
57,BURTON,STEVEN,G.,9938 DEER CREEK DRIVE,NaN,TAMPA,FL,33647,-2300.0,2008-03-05,22
111,Reich,Thomas,NaN,499 Park Ave,NaN,New York,NY,100221240,-2300.0,2008-08-28,34
58,BURTON,GLENN,M.,4404 CHARLESTON COURT,NaN,TAMPA,FL,336092620,-2300.0,2008-03-05,22
55,BUSH,ERIC,NaN,P.O. BOX 61046,NaN,DENVER,CO,802061046,-2300.0,2008-03-06,22


#### Q2.2 How do you do it in SQL?

##### First in ascending order.

In [19]:
out=make_query("SELECT * FROM contributors ORDER BY amount;")
make_frame(out, legend=contributor_cols).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,91,Kazor,Christopher,M,707 Spindletree ave,None,Naperville,IL,60565,-2592.0,2008-04-21,32
1,30,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,-2300.0,2007-08-14,20
2,52,BYINGTON,MARGARET,E.,2633 MIDDLEBORO LANE N.E.,None,GRAND RAPIDS,MI,495061254,-2300.0,2008-03-03,22
3,53,BYERS,BOB,A.,13170 TELFAIR AVENUE,None,SYLMAR,CA,913423573,-2300.0,2008-03-07,22
4,55,BUSH,KRYSTIE,None,P.O. BOX 61046,None,DENVER,CO,802061046,-2300.0,2008-03-06,22
5,56,BUSH,ERIC,None,P.O. BOX 61046,None,DENVER,CO,802061046,-2300.0,2008-03-06,22
6,57,BURTON,SUSAN,None,9338 DEER CREEK DRIVE,None,TAMPA,FL,336472286,-2300.0,2008-03-05,22
7,58,BURTON,STEVEN,G.,9938 DEER CREEK DRIVE,None,TAMPA,FL,33647,-2300.0,2008-03-05,22
8,59,BURTON,GLENN,M.,4404 CHARLESTON COURT,None,TAMPA,FL,336092620,-2300.0,2008-03-05,22
9,65,BURKE,DONALD,J.,12 LOMPOC,None,RANCHO SANTA MARGA,CA,926881817,-2300.0,2008-03-11,22


##### Now in descending order.

In [20]:
out=make_query("SELECT * FROM contributors ORDER by amount DESC")
make_frame(out, legend=contributor_cols).head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600.0,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,None,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,14,Altes,R.D.,None,8600 Moody Road,None,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,None,211 Long Island Drive,None,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,None,2550 Adamsbrooke Drive,None,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,2300.0,2007-08-14,20
6,34,Buck,Blaine,M,45 Eaton Ave,None,Camden,ME,48431752,2300.0,2007-09-30,20
7,136,ABRAMOWITZ,NIRA,None,411 HARBOR ROAD,None,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35
8,6,Akin,Mike,None,181 Baywood Lane,None,Monticello,AR,71655,1500.0,2007-05-18,16
9,10,Allen,John D.,None,1052 Cannon Mill Drive,None,North Augusta,SC,29860,1300.0,2007-06-29,16


### Q3: Find all distinct (distinct first name/last name pairs) contributors

Once we have chosen certain columns we might want to drop rows which have duplicate values for some of these columns..

Selecting a distinct set is useful for cleaning. Here, we might wish to focus on contributors rather than contributions and see how many distinct contributors we have. Of-course we might be wrong, some people have identical names. 

#### Q3.1 How do you do it in Pandas?

In [21]:
#Take a look at the first name and last name columns
dfcwci[['last_name','first_name']]

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
2,Ahrens,Don
3,Ahrens,Don
4,Akin,Charles
...,...,...
170,ABESHAUS,MERRILL
171,ABRAHAM,GEORGE
172,ABRAHAMSON,PETER
173,ABRAHAM,SALEM


This is how you drop duplicates in Pandas. Think of which of the duplicates might be dropped?

In [22]:
dfcwci[['last_name','first_name']].drop_duplicates()

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
4,Akin,Charles
5,Akin,Mike
6,Akin,Rebecca
...,...,...
169,ABERCROMBIE,DENIS
170,ABESHAUS,MERRILL
171,ABRAHAM,GEORGE
172,ABRAHAMSON,PETER


Let us now create the entire dataframe with the duplicates dropped.

In [23]:
### edTest(test_duplicates) ###
mask = dfcwci[['last_name','first_name']].duplicated()
print(mask[:10])
uniquedfcwci=dfcwci[~mask]
uniquedfcwci

0    False
1    False
2     True
3     True
4    False
5    False
6    False
7    False
8    False
9     True
dtype: bool


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
...,...,...,...,...,...,...,...,...,...,...,...
169,ABERCROMBIE,DENIS,NaN,11811 WATER OAK CT,NaN,MAGNOLIA,TX,773546270,500.0,2008-01-30,37
170,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,NaN,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37
171,ABRAHAM,GEORGE,NaN,P.O. BOX 1504,NaN,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37
172,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,NaN,CHICAGO,IL,606572207,50.0,2008-01-25,37


### Q3.2 How do you do it in SQL?

In [24]:
out=make_query("SELECT DISTINCT last_name, first_name FROM contributors ")
make_frame(out,['last_name', 'first_name'])

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
2,Akin,Charles
3,Akin,Mike
4,Akin,Rebecca
...,...,...
121,ABERCROMBIE,DENIS
122,ABESHAUS,MERRILL
123,ABRAHAM,GEORGE
124,ABRAHAMSON,PETER


Is this very ad-hoc duplicate removal mechanism a good cleaning strategy?

In [25]:
db.close()

---